In [2]:
!ls -l
import json

total 2360
-rw-r--r--+ 1 maningmn1987  staff   77444 Sep 29 16:44 CSE103_Fall14_Week2_problem13_part1.txt
-rw-r--r--+ 1 maningmn1987  staff  196948 Sep 29 16:44 CSE103_Fall14_Week2_problem13_part2.txt
-rw-r--r--+ 1 maningmn1987  staff  246614 Sep 29 16:44 CSE103_Fall14_Week2_problem13_part3.txt
-rw-r--r--+ 1 maningmn1987  staff  282073 Sep 29 16:44 CSE103_Fall14_Week2_problem13_part4.txt
-rw-r--r--+ 1 maningmn1987  staff  177009 Sep 29 16:44 CSE103_Fall14_Week2_problem13_part5.txt
-rw-r--r--+ 1 maningmn1987  staff   33019 Sep 24 08:23 Function pointers and multiprocess communication.ipynb
-rw-r--r--+ 1 maningmn1987  staff    5761 Sep 24 08:23 Load Libraries.ipynb
-rw-r--r--+ 1 maningmn1987  staff    2878 Sep 15 16:28 Problem2.3.json
-rw-r--r--+ 1 maningmn1987  staff   65469 Sep 15 16:28 Problem8Attempts.json
-rw-r--r--+ 1 maningmn1987  staff   17516 Oct  1 08:05 Process Data.ipynb
-rw-r--r--+ 1 maningmn1987  staff   23921 Sep 24 08:23 Week3.Prob8.Part1.json
-rw-r--r--+ 1 maningmn1987  

In [3]:

%cd ../filter_helpers/

/Users/maningmn1987/Documents/Study_material/CSE103_TA/Webwork_AdaptiveHints/src/servers/rest_server/filter_helpers


In [4]:
%%writefile classify_final_value.py









   




def classify_final_value(attempt_flat,answer_flat):
    """ Written by Yoav Freund, Sat Sep 19 17:09:48 PDT 2015
    classifies the final value as: correct,parse_error,int, not_int
    """
    
    att_final_value=attempt_flat[0]['val']
    ans_final_value=answer_flat[0]['val']
    
    if att_final_value==ans_final_value:
        return 'correct'
    if isinf(att_final_value):
        return "inf"
    if int(att_final_value)!=att_final_value:
        return "not int"
    else:
        return "int"

Overwriting classify_final_value.py


In [5]:
%%writefile answer_is_not_expression.py        
def answer_is_not_expression(answer_string, parse_tree, eval_tree, correct_string, correct_tree, correct_eval, user_vars):
    if len(att_eval_tree)==1:
        return "Please enter an expression, not just final numerical result"
    else:
        return ""

Overwriting answer_is_not_expression.py


In [6]:
def check_81(attempt,att_parse_tree,att_eval_tree,answer,answer_parse_tree,answer_eval_tree,variables):
    """ name date 
    description """
    attempt_class=classify_final_value(att_eval_tree,answer_eval_tree)
    if  attempt_class=='correct':
        return 'correct answer'
    if attempt_class=='int':
        return 'Can the answer be an integer number? [_____]{n}'
    
    hint=answer_is_not_expression(attempt,att_parse_tree,att_eval_tree,answer,answer_parse_tree,answer_eval_tree,variables)
    if  len(hint)!=0: 
        return hint

    return None

In [7]:
cd ../filter_helpers/

/Users/maningmn1987/Documents/Study_material/CSE103_TA/Webwork_AdaptiveHints/src/servers/rest_server/filter_helpers


In [30]:
att_eval_tree=  [[u'/', 0.451171875, [0, 6]], [462.0], [[u'^', 1024.0, [4, 6]], [2.0], [10.0]]]

In [24]:
%%writefile flatten.py
from numbers import Number
def flatten(L):
    if type(L)!=list:
        #print 'not list'
        return None
    elif len(L)==1 and isinstance(L[0],Number):
        #print 'list of one number'
        return [{'val':L[0]}]
    elif len(L)==3 and type(L[0])==unicode:
        if not isinstance(L[1],Number):
            print "expected number, but got %s, in %s"%(str(L[1]),str(L))
            return None
        if type(L[2])==list:
            if len(L[2])==2:
                if type(L[2][0])==int and type(L[2][1])==int:
                    return [{'val':L[1],'location':L[2]}]
        print "expected a tuple of the form (operation,value,[start,end])"
        print "instead got "+str(L)
        return None
    else:
        #print 'general list'
        answer=[]
        for item in L:
            fitem=flatten(item)
            if fitem==None:
                return None
            answer = answer+fitem
        return answer

Writing flatten.py


In [9]:
%%writefile find_common.py
def find_range(item,flat_tree):
    for entry in flat_tree:
        if item==entry['val']:
            return entry['location']
    return None
    
def find_common(attempt,att_flat,answer,ans_flat):
    "Find sub-expressions common to the attempt and the answer"

    att_set=[x['val'] for x in att_flat if x.has_key('val')]
    answer_set=[x['val'] for x in ans_flat if x.has_key('val')]

    intersection=[x for x in set(att_set) & set(answer_set) if (int(x)!=x) or x>100]
    
    if len(intersection)==0:
        return None
 
    print 'common subexpressions=',str(intersection)
    #print 'attempt=%s, att_eval_tree=%s\n variables=%s,answer=%s,answer_eval_tree=%s'\
    #    %(attempt,str(att_eval_tree),str(variables),answer,str(answer_eval_tree))

    for item in intersection:
        att_range=find_range(item,att_flat)
        ans_range=find_range(item,ans_flat)
        print "attempt subexpression %s is equivalent to answer subexpression %s"\
        %(attempt[att_range[0]:att_range[1]+2],answer[ans_range[0]:ans_range[1]+2])
 
        

Overwriting find_common.py


In [10]:
%%writefile substit.py
from string import replace
def substit(string,variables):
    for var in variables.keys():
        val=str(variables[var])
        string=string.replace(var,val)
    return string

Overwriting substit.py


In [199]:
#%%writefile main.py
file=open('CSE103_Fall14_Week2_problem13_part4.txt','r');
ID=file.readline()
answer=file.readline()
print 'answer=',answer
count =0
from substit import substit
from flatten import flatten
for line in file.readlines():
    count+=1
    if count>1: break
    params=json.loads(line);
    print params
    attempt,att_parse_tree,att_eval_tree,answer,answer_parse_tree,answer_eval_tree,variables = params
    print 'answer before substitution',answer
    answer=substit(answer,variables)
    print 'answer after substitution',answer
    attempt_flat=flatten(att_eval_tree)
    answer_flat =flatten(answer_eval_tree)
    print attempt_flat
    print answer_flat
    if attempt_flat==None or answer_flat==None:
        print 'Eval tree== None or parsing error ------------------------------'
        print str(params)
        break

    Class=classify_final_value(attempt_flat,answer_flat)
    if Class=='correct' or Class=='int':
        print Class
        continue

    print "attempt={:<40} answer={:<40}".format(attempt,answer)
    hint= find_common(attempt,attempt_flat,answer,answer_flat)
    

answer= 1-1/2^$coins

answer before substitution 1-1/2^$coins
answer after substitution 1-1/2^11.0
[{'location': [0, 8], 'val': 0.9990234375}, {'val': 1.0}, {'location': [4, 8], 'val': 0.0009765625}, {'val': 1.0}, {'location': [6, 8], 'val': 1024.0}, {'val': 2.0}, {'val': 10.0}]
[{'location': [0, 6], 'val': 0.99951171875}, {'val': 1.0}, {'location': [2, 6], 'val': 0.00048828125}, {'val': 1.0}, {'location': [4, 6], 'val': 2048.0}, {'val': 2.0}, {'val': 11.0}]
attempt=1 - 1/2^10                               answer=1-1/2^11.0                              
answer before substitution 1-1/2^$coins
answer after substitution 1-1/2^11.0
[{'location': [0, 8], 'val': 0.99951171875}, {'val': 1.0}, {'location': [4, 8], 'val': 0.00048828125}, {'val': 1.0}, {'location': [6, 8], 'val': 2048.0}, {'val': 2.0}, {'val': 11.0}]
[{'location': [0, 6], 'val': 0.99951171875}, {'val': 1.0}, {'location': [2, 6], 'val': 0.00048828125}, {'val': 1.0}, {'location': [4, 6], 'val': 2048.0}, {'val': 2.0}, {'val': 11.0

KeyError: 'location'

In [21]:
"{:<7}{:<51}{:<25}\n".format('this', 'that', 'and the other')

'this   that                                               and the other            \n'

In [193]:
%cd filter_lab

/Users/yoavfreund/projects/Webwork.Improvement.Project/AdaptiveHints/src/servers/rest_server/filter_lab


In [9]:
defined X

SyntaxError: invalid syntax (<ipython-input-9-56d3901bedac>, line 1)

In [43]:
from numpy import isinf,isn
isinf(float('inf'))

True

In [83]:
import numbers

In [88]:
isinstance('10',numbers.Number)

False

In [92]:
'abs'=='abd'

False

In [106]:
answer

u''

In [107]:
X=None
X==None

True